<a href="https://colab.research.google.com/github/felixzhao/title_catgories_classification/blob/main/2.2_preprocess_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# load packages

In [ ]:
import pandas as pd
import json
import pickle

load data
This work loads data from google drive. Please put the files into "trademe_data" folder under the root of google drive.

Then run the following code to mount the google drive in colab.

You can follow the guide. https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_path = "drive/MyDrive/trademe_data/"

Mounted at /content/drive


# load  data

In [ ]:
cat_raw = pd.read_csv(f"{root_path}categories.csv")
cat_raw.head(1)

,y_cat_id,cat_1,cat_2,cat_3
0,4957,construction & roading,health & safety,health & safety


# get level 3 hierarchy

In [ ]:
c3_dict = {}

c3_grouped = cat_raw.groupby(['cat_2', 'cat_1'])

for name, group in c3_grouped:
    # print(name)
    # print(group)
    # if len(group) > 1:
    #     print(name[0])
    #     print(group.cat_3.to_list())
    if len(group) > 1:
        c3_dict[name[0]] = group.cat_3.to_list()

c3_dict

{'accountants': ['business advisory services',
  'assistant accountants',
  'audit',
  'financial accounting & reporting',
  'general / ca',
  'management accountants',
  'tax'],
 'drivers & couriers': ['couriers', 'truck ht'],
 'nursing & midwifery': ['community & primary care',
  'critical care & emergency',
  'medical',
  'mental health',
  'paediatrics',
  'surgical',
  'theatre & recovery',
  'women’s health']}

In [ ]:
c1_dict = {}

c1_grouped = cat_raw.groupby(['cat_2', 'cat_3'])

for name, group in c1_grouped:
    if len(group) > 1:
        c1_dict[name[0]] = group.cat_1.to_list()

c1_dict

{'administration': ['healthcare', 'office & administration'],
 'analysts': ['banking, finance & insurance', 'accounting'],
 'architects': ['architecture', 'it'],
 'drafting': ['architecture', 'engineering'],
 'electrical': ['engineering', 'trades & services'],
 'health & safety': ['construction & roading', 'hr & recruitment'],
 'management': ['automotive',
  'accounting',
  'banking, finance & insurance',
  'customer service',
  'engineering',
  'transport & logistics',
  'healthcare',
  'hospitality & tourism',
  'it',
  'manufacturing & operations',
  'marketing, media & communications',
  'retail'],
 'project management': ['engineering', 'it'],
 'security': ['trades & services', 'it'],
 'supervisors & forepersons': ['construction & roading',
  'manufacturing & operations']}

# Save to file

In [ ]:
def write_to_file(file_name, dict_to_save):
    with open(file_name, "w") as file:
        json.dump(dict_to_save, file, indent=4)  

In [ ]:
write_to_file(f"{root_path}c3_dict.json", c3_dict)

In [ ]:
write_to_file(f"{root_path}c1_dict.json", c1_dict)

# Get label id mapping for each level

## Category Level 2

In [ ]:
cat_raw.head(1)

,y_cat_id,cat_1,cat_2,cat_3
0,4957,construction & roading,health & safety,health & safety


In [ ]:
c2_mapping = {}

c2_grouped = cat_raw.groupby(['cat_2'])

for name, group in c2_grouped:
    # print(name)
    # print(group)
    # if len(group) == 1:
    #     print(name)
    #     print(group.iloc[0].y_cat_id)
    if len(group) == 1:
        c2_mapping[name] = group.iloc[0].y_cat_id

print(len(c2_mapping))
list(c2_mapping.items())

149


[('account management', 5164),
 ('accounts administrators', 5007),
 ('accounts payable', 5004),
 ('accounts receivable', 5005),
 ('advertising', 763),
 ('air con. & refrigeration', 5182),
 ('area managers', 765),
 ('assistant store managers', 5177),
 ('au pairs & nannies', 748),
 ('automotive technician', 4960),
 ('bar staff & baristas', 5098),
 ('beautician', 6902),
 ('boat building', 6903),
 ('boilermakers & welders', 5184),
 ('brand & product management', 5145),
 ('building & carpentry', 5186),
 ('building services', 5057),
 ('business & systems analysts', 5114),
 ('business development manager', 5165),
 ('butchers & bakers', 5185),
 ('buying', 766),
 ('call centre', 733),
 ('caregiving', 5084),
 ('central government', 5079),
 ('chefs', 5100),
 ('civil & structural', 5058),
 ('cleaning', 5187),
 ('client services', 5024),
 ('commercial sales & leasing', 821),
 ('communications & pr', 5150),
 ('community & social services', 5085),
 ('consultancy & valuation', 840),
 ('corporate & ins

In [ ]:
c3_mapping = {}

c3_grouped = cat_raw.groupby(['cat_2', 'cat_1'])

for name, group in c3_grouped:
    # print(name)
    # print(group)
    # if len(group) > 1:
    #     print(name[0])
    #     print(dict(zip(group.y_cat_id, group.cat_3)))
    if len(group) > 1:
        c3_mapping[name[0]] = dict(zip(group.y_cat_id, group.cat_3))

print(len(c3_mapping))
list(c3_mapping.items())[:3]

3


[('accountants',
  {621: 'business advisory services',
   6882: 'assistant accountants',
   6883: 'audit',
   6884: 'financial accounting & reporting',
   6885: 'general / ca',
   6886: 'management accountants',
   6887: 'tax'}),
 ('drivers & couriers', {6908: 'couriers', 6909: 'truck ht'}),
 ('nursing & midwifery',
  {900: 'community & primary care',
   904: 'critical care & emergency',
   906: 'medical',
   907: 'mental health',
   910: 'paediatrics',
   916: 'surgical',
   927: 'theatre & recovery',
   929: 'women’s health'})]

In [ ]:
c1_mapping = {}

c1_grouped = cat_raw.groupby(['cat_2', 'cat_3'])

for name, group in c1_grouped:
    # print(name)
    # print(group)
    # if len(group) > 1:
    #     print(name[0])
    #     print(dict(zip(group.y_cat_id, group.cat_3)))
    if len(group) > 1:
        c1_mapping[name[0]] = dict(zip(group.y_cat_id, group.cat_1))

print(len(c1_mapping))
list(c1_mapping.items())[:3]

10


[('administration', {5083: 'healthcare', 5156: 'office & administration'}),
 ('analysts', {5022: 'banking, finance & insurance', 6889: 'accounting'}),
 ('architects', {5034: 'architecture', 5113: 'it'})]

In [ ]:
write_to_file(f"{root_path}c1_mapping.json", c1_mapping)

In [ ]:
def write_to_file_pickle(file_name, dict_to_save):
    with open(file_name, "wb") as file:
        pickle.dump(dict_to_save, file)  

In [ ]:
write_to_file_pickle(f"{root_path}c2_mapping.json", c2_mapping)

In [ ]:
write_to_file(f"{root_path}c3_mapping.json", c3_mapping)

# load from file to dict

In [ ]:
def read_from_file_pickle(file_name):
    res = None
    with open(file_name, "rb") as file:
        res = pickle.load(file) 
    return res 

In [ ]:
load_dict = read_from_file_pickle(f"{root_path}c2_mapping.json")
print(type(load_dict))

<class 'dict'>
